In [5]:
import pickle as pkl
import numpy as np
import os
# import jax
from matplotlib import pyplot as plt
import json

In [7]:
# INPUT_FOLDER = "/Users/evanpan/Documents/GitHub/EyeIK/Juggling/juggling_TAP_output"
# OUTPUT_FOLDER = "/Users/evanpan/Documents/GitHub/EyeIK/Juggling/juggling_TAP_output"
INPUT_FOLDER = "../Juggling/juggling_TAP_output"
OUTPUT_FOLDER = "./reading_output"
file_name = "reading_ver1"

# print the root of current exec

In [8]:
file_path = os.path.join(INPUT_FOLDER, file_name + ".pkl")
output_path = os.path.join(OUTPUT_FOLDER, file_name + ".pkl")

# generate the 2D position of the gaze when reading a book

In [9]:
page_half_width = 10
page_height = 10
lines_per_page = 10
speed_to_finish_row = 1 # this is 2 seconds
std_speed = 0.2 # the reading could be fast or slow
fps = 30


import math
# time to finish each page
time_per_page = []
for i in range(0, lines_per_page):
    time_per_page.append(speed_to_finish_row + np.random.normal(0, std_speed))
# total time to finish this page
total_time = sum(time_per_page)
# get some important details
height_per_row = page_height / lines_per_page

# the time_array
time_arr = np.linspace(0, total_time, int(total_time * fps))

# get the index of each time_stamp:
index_arr = []
index_i = 0
so_far = 0
for i in range(0, time_arr.shape[0]):
    if index_i < len(time_per_page):
        if time_arr[i] > so_far + time_per_page[index_i]:
            so_far += time_per_page[index_i]
            index_i += 1
    index_arr.append(index_i)
    
    
# get the position of each ball
ball_positions = []
for i in range(0, lines_per_page):
    # generate a trajectory for each row, going from -page_half_width to page_half_width, at a speed of 2 * page_half_width / time_per_page[i]
    x_speed = 2 * page_half_width / time_per_page[i]
    # starting position
    sphere_height = i * height_per_row
    starting_x = -page_half_width
    # delta time
    dt = 1 / fps
    
    # get the starting time_step
    starting_time_step = np.sum(time_per_page[0:i])
    starting_time_index = int(math.floor(starting_time_step * fps))
    total_time_per_line = time_per_page[i]
    
    # obtain the trajectory of the ball on that line
    trajectory_line_i = np.ones((time_arr.shape[0], 2)) * -page_half_width
    trajectory_line_i[:, 0] = -page_half_width
    trajectory_line_i[:, 1] = sphere_height
    for t in range(0, int(total_time_per_line * fps)):
        trajectory_line_i[starting_time_index + t, 0] = starting_x + x_speed * t * dt
        trajectory_line_i[starting_time_index + t, 1] = sphere_height
    # fill the rest of the trajectory with the last position
    for t in range(starting_time_index + int(total_time_per_line * fps), time_arr.shape[0]):
        trajectory_line_i[t, 0] = trajectory_line_i[starting_time_index + int(total_time_per_line * fps) - 1, 0]
        trajectory_line_i[t, 1] = trajectory_line_i[starting_time_index + int(total_time_per_line * fps) - 1, 1]
    ball_positions.append(trajectory_line_i)

output_json = {}
# make sure each list is a list
output_json["time"] = time_arr.tolist()
for i in range(0, lines_per_page):
    output_json[f"ball_{i}_positions"] = (-ball_positions[i]).tolist()
output_json["look_at_indices"] = index_arr
with open(output_path, 'w') as f:
    json.dump(output_json, f)



balls = []

time = 0
row_i = 0 # the vertical position of the gaze on the page
col_x = 0 # the horizontal position of the gaze on the page

# while row_i < lines_per_page:
    
 






